In [ ]:
import os
import re
import string

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem import (SnowballStemmer)

from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

In [ ]:
from torch.utils.data import DataLoader
import torch

import torch
import numpy as np
import pandas as pd
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from timeit import default_timer as timer
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
import math
from torch import Tensor
import pickle
from torch.autograd import Variable

In [ ]:
item = torch.load('C://Users//user//Desktop//王茂田//item.pt')
duration=torch.load('C://Users//user//Desktop//王茂田//duration.pt')
interval=torch.load('C://Users//user//Desktop//王茂田//interval.pt')

In [ ]:
item = item.numpy().tolist()
duration = duration.numpy().tolist()
interval = interval.numpy().tolist()

In [ ]:
target_item=[]
target_duration=[]
target_interval=[]

for i in range (len(item)):
    target_item=target_item+(item[i][-1:])
    
for i in range (len(duration)):
    target_duration=target_duration+(duration[i][-1:])
    
for i in range (len(interval)):
    target_interval=target_interval+(interval[i][-1:])

In [ ]:
input_item=[]
input_duration=[]
input_interval=[]

for i in range (len(item)):
    input_item.append(item[i][:-1])
    
for i in range (len(duration)):
    input_duration.append(duration[i][:-1])
    
for i in range (len(interval)):
    input_interval.append(interval[i][:-1])

In [ ]:
lists_of_lists1=(input_item,target_item,input_duration,target_duration,input_interval,target_interval)

In [ ]:
import random
l1=random.sample(range(0,len(lists_of_lists1[0])),int(len(lists_of_lists1[0])*0.7))
train_data=[[],[],[],[],[],[]]
test_data=[[],[],[],[],[],[]]
for i in range(len(lists_of_lists1[0])):
    if i in set(l1):
        train_data[0].append(lists_of_lists1[0][i])
        train_data[1].append(lists_of_lists1[1][i])
        train_data[2].append(lists_of_lists1[2][i])
        train_data[3].append(lists_of_lists1[3][i])
        train_data[4].append(lists_of_lists1[4][i])
        train_data[5].append(lists_of_lists1[5][i])
    else:
        test_data[0].append(lists_of_lists1[0][i])
        test_data[1].append(lists_of_lists1[1][i])
        test_data[2].append(lists_of_lists1[2][i])
        test_data[3].append(lists_of_lists1[3][i])
        test_data[4].append(lists_of_lists1[4][i])
        test_data[5].append(lists_of_lists1[5][i])

In [ ]:
train_data[0] = torch.LongTensor(train_data[0]).to(torch.int64)
train_data[1] = torch.LongTensor(train_data[1]).to(torch.int64)
train_data[2] = torch.LongTensor(train_data[2]).to(torch.int64)
train_data[3] = torch.LongTensor(train_data[3]).to(torch.int64)
train_data[4] = torch.LongTensor(train_data[4]).to(torch.int64)
train_data[5] = torch.LongTensor(train_data[5]).to(torch.int64)


test_data[0] = torch.LongTensor(test_data[0]).to(torch.int64)
test_data[1] = torch.LongTensor(test_data[1]).to(torch.int64)
test_data[2] = torch.LongTensor(test_data[2]).to(torch.int64)
test_data[3] = torch.LongTensor(test_data[3]).to(torch.int64)
test_data[4] = torch.LongTensor(test_data[4]).to(torch.int64)
test_data[5] = torch.LongTensor(test_data[5]).to(torch.int64)

In [ ]:
import torch.utils.data as data_utils
train = data_utils.TensorDataset(train_data[0], train_data[1],train_data[2],train_data[3], train_data[4],train_data[5])
test = data_utils.TensorDataset(test_data[0],test_data[1], test_data[2],test_data[3],test_data[4], test_data[5])

In [ ]:
#載入dataloader，並選擇batch_size
batch_size = 16
train_loader = DataLoader(train, batch_size=batch_size, shuffle=False, drop_last=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False, drop_last=True)

In [ ]:
dataiter = iter(train_loader)
a = dataiter.next()
a

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class RNNModel(nn.Module):

    def __init__(self, embedded_size,hidden_num):
        super(RNNModel, self).__init__()
        # item_num, embedding_size
        self.embedded_size = embedded_size
        self.hidden_size = hidden_num
        
        self.it_embedding = nn.Embedding(776, self.embedded_size)
        self.du_embedding = nn.Embedding(21, self.embedded_size)
        self.int_embedding = nn.Embedding(21, self.embedded_size)
       
        
        #主模型 (embedding_size, hidden_size)
        self.RNN_layer = nn.RNN(self.embedded_size*3,self.hidden_size*3, batch_first=True)
        #最終輸出層(Crossentropy)
        self.out_it = nn.Linear(self.hidden_size*3, 776)
        self.out_du = nn.Linear(self.hidden_size*3, 21)
        self.out_int = nn.Linear(self.hidden_size*3, 21)

        
    def forward(self, input):
        
        items = input['item'] 
        target_items = input['target_item']
        durations = input['duration'] 
        target_durations = input['target_duration']
        intervals = input['interval'] 
        target_intervals = input['target_interval'] 
        
        embedded_item = self.it_embedding(items)
   
        embedded_duration = self.du_embedding(durations)
        embedded_interval = self.int_embedding(intervals)
        x = torch.cat((embedded_item,embedded_duration,embedded_interval),dim=2)
        

      
        output, _ = self.RNN_layer(x)
        output = output[:, -1, :]
        output1 = self.out_it(output)
        output2 = self.out_du(output)
        output3 = self.out_int(output)
        
        return output1,output2,output3
      
        

In [ ]:
# input_num, hidden_num, output_num
model = RNNModel(128,128)
model

In [ ]:
#設定訓練步驟
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.99), eps=1e-8)
epochs = 30

train_losses = []

logs = []
prod_all = []
label_all = []

In [ ]:


import torch
for epoch in range(epochs):
    co = 0

    correct_duration=0
    correct_interval=0
  
    train_loss_sum = 0.0
    
    train_loss = list()

    start_time = timer()
    model.train()
   
    b = []
    m, t= 0,0
    re3_it, re5_it, re10_it,  ra_it = 0, 0, 0, 0
    re2_du, re3_du, re5_du,  ra_du = 0, 0, 0, 0
    re2_int, re3_int, re5_int,  ra_int = 0, 0, 0, 0
    m3_it,m5_it,m10_it=0,0,0
    nd3_it,nd5_it,nd10_it=0,0,0
    
    m2_du,m3_du,m5_du=0,0,0
    nd2_du,nd3_du,nd5_du=0,0,0
    
    m2_int,m3_int,m5_int=0,0,0
    nd2_int,nd3_int,nd5_int=0,0,0
    
    
    
    for batch, data in enumerate(train_loader):
        #將input的tuple分成三個部分
        
        input = { 'item': data[0],
                  'target_item': data[1],
                  'duration': data[2],
                  'target_duration': data[3],
                  'interval': data[4],
                  'target_interval': data[5],
                 }
        logits_it,logits_du,logits_int = model(input)
        l_it = logits_it
        l_du = logits_du
        l_int = logits_int
        tgt_it = input['target_item']
        tgt_du = input['target_duration']
        tgt_int = input['target_interval']
        
       
        # CROSS比較train跟target
        loss_it = loss_fn(l_it, tgt_it)
        loss_du = loss_fn(l_du, tgt_du)
        loss_int = loss_fn(l_int, tgt_int)
        
        loss = 0.4*loss_it + 0.3*loss_du +0.3*loss_int
        loss.backward()
        optimizer.step()
        train_loss_sum += loss.item()
       
        
        
 
    train_loss = train_loss_sum / len(train_loader)
    train_time = round(timer() - start_time)
    model.eval()
    torch.cuda.empty_cache()
    
    import heapq
    
   

    for batch, data in tqdm(enumerate(test_loader)):   
        input = { 'item': data[0],
                  'target_item': data[1],
                  'duration': data[2],
                  'target_duration': data[3],
                  'interval': data[4],
                  'target_interval': data[5],
                 }

        logits_it,logits_du,logits_int = model(input)
   
        l_it = logits_it
        l_du = logits_du
        l_int = logits_int
        tgt_it = input['target_item']
        tgt_du = input['target_duration']
        tgt_int = input['target_interval']
        
   
        
        l_it =  l_it.cpu().data.numpy()
        l_du =  l_du.cpu().data.numpy()
        l_int =  l_int.cpu().data.numpy()
        ranks_it = 0
        ranks3_it = 0
        ranks5_it = 0
        ranks10_it = 0
        ndcg3_it=0
        ndcg5_it=0
        ndcg10_it=0
        
        ranks_du = 0
        ranks2_du = 0
        ranks3_du = 0
        ranks5_du = 0
        ndcg2_du=0
        ndcg3_du=0
        ndcg5_du=0
        
        ranks_int = 0
        ranks2_int = 0
        ranks3_int = 0
        ranks5_int = 0
        ndcg2_int=0
        ndcg3_int=0
        ndcg5_int=0
        
        
        
      
       #####item
        # 將前k個結果取出做比較
        for n in range(len(l_it)):
             
            r3= heapq.nlargest(3, range(len(l_it[n])), l_it[n].take) 

            
            for i in range(len(r3)):
                re3_it += ((r3[i]) == tgt_it[n]).float().sum()
                
            r5= heapq.nlargest(5, range(len(l_it[n])), l_it[n].take)
       
            for i in range(len(r5)):
                re5_it += ((r5[i]) == tgt_it[n]).float().sum()
                
                
            r10= heapq.nlargest(10, range(len(l_it[n])), l_it[n].take)
          
            for i in range(len(r10)):
                re10_it += ((r10[i]) ==tgt_it[n]).float().sum()
            
          #MRR3
            mr3 = r3
            for i in range(len(mr3)):
                if (mr3[i]) == tgt_it[n]:
                    rank3 = i + 1
                    rranks3 = 1/rank3

                    m3_it += rranks3
                    nndcg3=(1/(np.log2(rank3+1)))
                    nd3_it+=nndcg3
        #MRR
            mr5 = r5

            for i in range(len(mr5)):
               
                if (mr5[i]) == tgt_it[n]:
                    rank5 = i + 1
                    rranks5 = 1/rank5
                    m5_it += rranks5
                    nndcg5=(1/(np.log2(rank5+1)))
                    nd5_it+=nndcg5
    
    
    #MRR
            mr10 = r10

            for i in range(len(mr10)):
                if (mr10[i]) == tgt_it[n]:
                    rank10 = i + 1
                   
                    rranks10 = 1/rank10
                    m10_it += rranks10
                    nndcg10=1/(np.log2(rank10+1))
                    nd10_it+=nndcg10
        
         #####duration
        # 將前k個結果取出做比較      
        
        
        for n in range(len(l_du)):
            
            r1= heapq.nlargest(1,  range(len(l_du[n])), l_du[n].take) 
            
            
            for i in range(len(r1)):
                correct_duration += (((r1[i]) == tgt_du[n]).float().sum())
            
            
            r2= heapq.nlargest(2,  range(len(l_du[n])), l_du[n].take) 
                
      
            for i in range(len(r2)):
                re2_du += (((r2[i]) == tgt_du[n]).float().sum())
            
            
            r3= heapq.nlargest(3,  range(len(l_du[n])), l_du[n].take) 
         
            for i in range(len(r3)):
                re3_du += (((r3[i]) == tgt_du[n]).float().sum())
            
            r5= heapq.nlargest(5,  range(len(l_du[n])), l_du[n].take) 
            
            for i in range(len(r5)):
                re5_du += (((r5[i]) ==tgt_du[n]).float().sum())
                
           
                           
          #MRR3
            mr2 = r2
            for i in range(len(mr2)):
                if ((mr2[i]) == tgt_du[n]):
                    rank2 = i + 1
                    rranks2 = 1/rank2

                    m2_du += rranks2
                    nndcg2=(1/(np.log2(rank2+1)))
                    nd2_du+=nndcg2
        #MRR
            mr3 = r3

            for i in range(len(mr3)):
               
                if ((mr3[i]) == tgt_du[n]):
                    rank3 = i + 1
                    rranks3 = 1/rank3
                    
                    m3_du += rranks3
                    nndcg3=(1/(np.log2(rank3+1)))
                    nd3_du+=nndcg3
    

    #MRR
            mr5 = r5

            for i in range(len(mr5)):
                if ((mr5[i]) == tgt_du[n]):
                    rank5 = i + 1
                   
                    rranks5 = 1/rank5
                    m5_du += rranks5
                    nndcg5=1/(np.log2(rank5+1))
                    nd5_du+=nndcg5
       
                    
                      
       
           #####interval
        # 將前k個結果取出做比較  
        for n in range(len(l_int)):
            
            
            r1= heapq.nlargest(1,  range(len(l_int[n])), l_int[n].take) 
           
            for i in range(len(r1)):
                correct_interval += (((r1[i]) == tgt_int[n]).float().sum())
            
            
            r2= heapq.nlargest(2,  range(len(l_int[n])), l_int[n].take) 
            
            
            for i in range(len(r2)):
                re2_int += (((r2[i])) == tgt_int[n]).float().sum()
                
            r3= heapq.nlargest(3,  range(len(l_int[n])), l_int[n].take) 
            
            for i in range(len(r3)):
                re3_int += (((r3[i])) == tgt_int[n]).float().sum()
                
            r5= heapq.nlargest(5,  range(len(l_int[n])), l_int[n].take) 
            
     
            for i in range(len(r5)):
                re5_int += (((r5[i])) == tgt_int[n]).float().sum()
            
          #MRR3
            mr2 = r2
            for i in range(len(mr2)):
                if (((mr2[i])) == tgt_int[n]):
                    rank2 = i + 1
                    rranks2 = 1/rank2

                    m2_int += rranks2
                    nndcg2=(1/(np.log2(rank2+1)))
                    nd2_int+=nndcg2
        #MRR
            mr3 = r3

            for i in range(len(mr3)):
               
                if ((mr3[i])) == tgt_int[n]:
                    rank3 = i + 1
                    rranks3 = 1/rank3
                    m3_int += rranks3
                    nndcg3=(1/(np.log2(rank3+1)))
                    nd3_int+=nndcg3
    
 
    #MRR
            mr5 = r5

            for i in range(len(mr5)):
                if ((mr5[i])) == tgt_int[n]:
                    rank5 = i + 1
                   
                    rranks5 = 1/rank5
                    m5_int += rranks5
                    nndcg5 = 1/(np.log2(rank5+1))
                    nd5_int+=nndcg5  
        
        
        
        
        
        
       
   
    
    recall3_it = (re3_it*100)/(len(test_data[0]))
    
    recall5_it = (re5_it*100)/(len(test_data[0]))
    recall10_it = (re10_it*100)/(len(test_data[0]))
    mrr3_it = (m3_it*100)/(len(test_data[0]))
    mrr5_it = (m5_it*100)/(len(test_data[0]))
    mrr10_it = (m10_it*100)/(len(test_data[0]))
    n3_it=(nd3_it*100)/(len(test_data[0]))
    n5_it=(nd5_it*100)/(len(test_data[0]))
    n10_it=(nd10_it*100)/(len(test_data[0]))
   
        
        
    recall2_du = (re2_du*100)/(len(test_data[0]))
    recall3_du = (re3_du*100)/(len(test_data[0]))
    recall5_du = (re5_du*100)/(len(test_data[0]))
    mrr2_du = (m2_du*100)/(len(test_data[0]))
    mrr3_du = (m3_du*100)/(len(test_data[0]))
    mrr5_du = (m5_du*100)/(len(test_data[0]))
    n2_du=(nd2_du*100)/(len(test_data[0]))
    n3_du=(nd3_du*100)/(len(test_data[0]))
    n5_du=(nd5_du*100)/(len(test_data[0]))
    
        
        
    recall2_int = (re2_int*100)/(len(test_data[0]))
    recall3_int = (re3_int*100)/(len(test_data[0]))
    recall5_int = (re5_int*100)/(len(test_data[0]))
    mrr2_int = (m2_int*100)/(len(test_data[0]))
    mrr3_int = (m3_int*100)/(len(test_data[0]))
    mrr5_int = (m5_int*100)/(len(test_data[0]))
    n2_int=(nd2_int*100)/(len(test_data[0]))
    n3_int=(nd3_int*100)/(len(test_data[0]))
    n5_int=(nd5_int*100)/(len(test_data[0]))

        
    accuracy_du = 100 * correct_duration/(len(test_data[0]))
    accuracy_int = 100 * correct_interval/(len(test_data[0]))
        
        
        
    log = f"Accuracy_Duration: {accuracy_du:.5f} Accuracy_Interval: {accuracy_int:.5f} HR3_Item: {recall3_it:.5f} HR5_Item: {recall5_it:.5f} HR10_Item: {recall10_it:.5f} HR2_du: {recall2_du:.5f} HR3_du: {recall3_du:.5f} HR5_du: {recall5_du:.5f} HR2_int: {recall2_int:.5f} HR3_int: {recall3_int:.5f} HR5_int: {recall5_int:.5f}  MRR3_item: {mrr3_it:.10f}  MRR5_item: {mrr5_it:.10f}  MRR10_item: {mrr10_it:.10f} MRR2_du: {mrr2_du:.10f}  MRR3_du: {mrr3_du:.10f}  MRR5_du: {mrr5_du:.10f} MRR2_int: {mrr2_int:.10f}  MRR3_int: {mrr3_int:.10f}  MRR5_int: {mrr5_int:.10f}  NDCG3_it: {n3_it:.10f}  NDCG5_it: {n5_it:.10f}  NDCG10_it: {n10_it:.10f}  NDCG2_du: {n2_du:.10f}  NDCG3_du: {n3_du:.10f}  NDCG5_du: {n5_du:.10f}  NDCG2_int: {n2_int:.10f}  NDCG3_int: {n3_int:.10f}  NDCG5_int: {n5_int:.10f} "
    print("train_loss_sum:",train_loss_sum)
    print(log)
    
   # print(m/t)